## 2. Evidence - Interpretability QAS Measurements.

Now we proceed to gather data about the Interpretability of the model, for the corresponding scenario.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(
    store_path, exist_ok=True
)  # Ensure we are creating the folder if it is not there.

set_context("OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

Define different folders that will be used as input or output for the data gathering process.

In [2]:
from pathlib import Path

# The path at which datasets are stored
DATASETS_DIR = Path.cwd() / "data"

# Path where the model files are stored.
MODELS_DIR = Path.cwd() / "model"

# The path at which media is stored
MEDIA_DIR = Path.cwd() / "media"
os.makedirs(MEDIA_DIR, exist_ok=True)

In [3]:
model_filename = (
    MODELS_DIR / "model_f3_a.json"
)  # The json file of the model to load
weights_filename = MODELS_DIR / "model_f_a.h5"  # The weights file for the model

In [4]:
from model_analysis import *

# Load the model/
loaded_model = load_model(model_filename, weights_filename)

In [ ]:
# Load and show the image.

flower_img = "flower3.jpg"  # Filename of flower image to use, public domain image adapted from: https://commons.wikimedia.org/wiki/File:Beautiful_white_flower_in_garden.jpg
flower_idx = (
    42  # Classifier index of associated flower (see OxfordFlower102Labels.csv)
)

im = read_image(os.path.join(DATASETS_DIR, flower_img))

plt.imshow(im)
plt.axis("off")
plt.show()

In [6]:
predictions = run_model(im, loaded_model)

baseline, alphas = generate_baseline_and_alphas()

In [7]:
interpolated_images = interpolate_images(
    baseline=baseline, image=im, alphas=alphas
)

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
    i += 1
    plt.subplot(1, len(alphas[0::10]), i)
    plt.title(f"alpha: {alpha:.1f}")
    plt.imshow(image)
    plt.axis("off")

plt.tight_layout()

In [ ]:
path_gradients = compute_gradients(
    loaded_model=loaded_model,
    images=interpolated_images,
    target_class_idx=flower_idx,
)
print(path_gradients.shape)

ig = integral_approximation(gradients=path_gradients)
print(ig.shape)

In [ ]:
ig_attributions = integrated_gradients(
    baseline=baseline,
    image=im,
    target_class_idx=flower_idx,
    loaded_model=loaded_model,
    m_steps=240,
)
print(ig_attributions.shape)

In [ ]:
fig = plot_img_attributions(
    image=im,
    baseline=baseline,
    target_class_idx=flower_idx,
    loaded_model=loaded_model,
    m_steps=240,
    cmap=plt.cm.inferno,
    overlay_alpha=0.4,
)

plt.savefig(MEDIA_DIR / "attributions.png")

In [12]:
from mlte.measurement.external_measurement import ExternalMeasurement
from mlte.value.types.image import Image

# Save to MLTE store.
img_collector = ExternalMeasurement("image attributions", Image)
img = img_collector.ingest(MEDIA_DIR / "attributions.png")
img.save(force=True)